In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [4]:
labels = df[1].values

In [5]:
type(labels)

numpy.ndarray

In [6]:
texts = df[0].values.tolist()

In [7]:
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification

In [8]:
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
# xlarge version => "albert-xlarge-v2"
# xxlarge version => "albert-xxlarge-v2"

In [9]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(labels)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, y_one_hot, test_size=0.2, random_state=0)

In [11]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

In [12]:
model = TFAlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
# xlarge version => "albert-xlarge-v2"
# xxlarge version => "albert-xxlarge-v2"

All model checkpoint layers were used when initializing TFAlbertForSequenceClassification.

Some layers of TFAlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
optimizer = tf.keras.optimizers.Adam(2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
checkpoint_filepath = "./checkpoints/checkpoint_albert_en"
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_best_only=True, save_weights_only=True)

In [15]:
model.fit(dict(X_train_tokenized), y_train, epochs=10, batch_size=128, validation_split=0.1, callbacks=[es, mc])

Epoch 1/10
39/39 [==============================] - 601s 15s/step - loss: 0.5098 - accuracy: 0.7670 - val_loss: 0.3783 - val_accuracy: 0.8538
Epoch 2/10
39/39 [==============================] - 566s 15s/step - loss: 0.3623 - accuracy: 0.8557 - val_loss: 0.3885 - val_accuracy: 0.8430
Epoch 3/10
39/39 [==============================] - 570s 15s/step - loss: 0.3136 - accuracy: 0.8810 - val_loss: 0.3762 - val_accuracy: 0.8430
Epoch 4/10
39/39 [==============================] - 577s 15s/step - loss: 0.2806 - accuracy: 0.8940 - val_loss: 0.3608 - val_accuracy: 0.8520
Epoch 5/10
39/39 [==============================] - 570s 15s/step - loss: 0.2477 - accuracy: 0.9139 - val_loss: 0.3556 - val_accuracy: 0.8538
Epoch 6/10
39/39 [==============================] - 570s 15s/step - loss: 0.2217 - accuracy: 0.9237 - val_loss: 0.3631 - val_accuracy: 0.8520
Epoch 7/10
39/39 [==============================] - 593s 15s/step - loss: 0.1992 - accuracy: 0.9344 - val_loss: 0.3666 - val_accuracy: 0.8574
Epoch 

In [16]:
model.load_weights(checkpoint_filepath)
model.evaluate(dict(X_test_tokenized), np.array(y_test))

44/44 [==============================] - 56s 1s/step - loss: 0.3439 - accuracy: 0.8642


[0.3438771069049835, 0.8641618490219116]

In [17]:
y_preds = model.predict(dict(X_test_tokenized))
prediction_probs = tf.nn.softmax(y_preds.logits,axis=1).numpy()
y_predictions = np.argmax(prediction_probs, axis=1)
y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

44/44 [==============================] - 59s 1s/step
              precision    recall  f1-score   support

           0       0.89      0.83      0.86       688
           1       0.84      0.90      0.87       696

    accuracy                           0.86      1384
   macro avg       0.87      0.86      0.86      1384
weighted avg       0.87      0.86      0.86      1384

